##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [2]:
%pip install pandas 
%pip install matplotlib
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [128]:
# Can have as many cells as you want for code
import pandas as pd
filepath = "./data/catB_train.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file



df = pd.read_parquet(filepath)


cols_df1 = df.columns[:8]  
cols_df2 = df.columns[8:18]  
cols_df3 = df.columns[18:24] 
cols_df4 = df.columns[24:34]  
cols_df5 = df.columns[34:41] 
cols_df6 = df.columns[41:158]  
cols_df7 = df.columns[158:272] 
cols_df8 = df.columns[272:] 

df1 = df[cols_df1]
df2 = df[cols_df2]
df3 = df[cols_df3]
df4 = df[cols_df4]
df5 = df[cols_df5]
df6 = df[cols_df6]
df7 = df[cols_df7]
df8 = df[cols_df8]

df1 = df1.drop(columns=df1.columns[:2])
df3 = df3[df3.columns[2:3]]
df3 = df3[df3.columns[2:3]]
df4 = df4.iloc[:, [0, 1, 2, 8, 9]]
df6 = df6.drop(columns=df6.columns[96:103])
df6 = df6.astype('float')
df7_1 = df7[df7.columns[78:85]]
df7_2 = df7[df7.columns[110:135]]
df7 = pd.concat([df7_1, df7_2], axis=1)


#combining columns in cat 6 based on ape, sumins and prempaid

ape_av = df6.columns[0:30].to_list() + df6.columns[89:96].to_list()
df6['av_ape'] = df6[ape_av].mean(axis=1)
df6 = df6.drop(columns=ape_av)

sumins_av = df6.columns[0:29].to_list() + df6.columns[59:67].to_list()
df6['av_sumins'] = df6[sumins_av].mean(axis=1)
df6 = df6.drop(columns=sumins_av)

prempaid_av = df6.columns[0:36]
df6['av_prempaid'] = df6[prempaid_av].mean(axis=1)
df6 = df6.drop(columns=prempaid_av)

df6.head


<bound method NDFrame.head of            av_ape     av_sumins   av_prempaid
19550   46.972973     37.837838   2558.722222
4600     0.000000      0.000000      0.000000
13337    0.000000      0.000000      0.000000
15074    0.000000      0.000000      0.000000
19724   18.810811  21621.621622   1024.666667
...           ...           ...           ...
11284    0.000000      0.000000      0.000000
11964  200.432432   6486.486486  10918.000000
5390     0.000000      0.000000      0.000000
860      1.945946  10810.810811    106.000000
15795    0.000000    270.270270      0.000000

[17992 rows x 3 columns]>

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [10]:
###...code...###

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [11]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [12]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

PermissionError: [Errno 1] Operation not permitted

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!